<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220102_sm_know_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install gluonnlp
# !pip install mxnet
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
# !pip install transformers
# !pip install sentencepiece
# !pip install kobert-transformers
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-3cxbdpqq/kobert-tokenizer_d289d28917de4f1dbcb658f324c019dc
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-3cxbdpqq/kobert-tokenizer_d289d28917de4f1dbcb658f324c019dc


In [2]:
# import torch
# from torch import nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
# from kobert.utils import get_tokenizer
# from kobert.pytorch_kobert import get_pytorch_kobert_model
# from transformers import AdamW
# from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [3]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings

In [4]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [33]:
warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('/content/gdrive/MyDrive/know/train/*.csv'))]
know_test = [pd.read_csv(path) for path in sorted(glob('/content/gdrive/MyDrive/know/test/*.csv'))]

In [34]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [35]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [36]:
def text_to_token(df:pd.DataFrame):
  input_ids = list()

  for i in range(len(df)):
    temp = tokenizer.encode_plus(df['text'].loc[i], add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask = True, 
            return_token_type_ids=True,
            truncation=True)
    input_ids.append(temp['input_ids'])

  return input_ids

In [37]:
years = ['2017', '2018', '2019', '2020']

rf_predicts = []

for year, train, test in zip(years, know_train, know_test):
    print(year)
    cols = []
    for col in train.columns:
        if col == 'ID':
            continue
        try:
            train[col] = train[col].map(int)
        except:
            train[col] =  train[col].map(str)
            cols.append(col)
    text = ''
    for col in cols:
      text += train[col] + ' '
    train['text'] = text
    train['input_ids'] = text_to_token(train)
    max_len = int((train['input_ids'].map(len).mean() + train['input_ids'].map(len).max())/3)
    def ids_to_trunc(text:list):
      if len(text) < max_len:
        return text
      else:
        return text[:max_len]

    train['input_ids'] = train['input_ids'].map(ids_to_trunc)
    for sentence in train['input_ids']: # 각 문장에 대해서
      while len(sentence) < max_len: # max_len보다 작으면
          sentence.append(0)
    train = pd.concat([train, pd.DataFrame(train['input_ids'].to_list(), columns=range(max_len))], axis=1)
    feature = train.columns.drop(['idx', 'knowcode', 'text', 'input_ids'])
    feature = feature.drop(cols)
    model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)
    model.fit(train[feature], train['knowcode'])

    for col in cols:
      test[col] =  test[col].map(str)

    text = ''
    for col in cols:
      text += test[col] + ' '
    test['text'] = text
    test['input_ids'] = text_to_token(test)
    def ids_to_trunc(text:list):
      if len(text) < max_len:
        return text
      else:
        return text[:max_len]

    test['input_ids'] = test['input_ids'].map(ids_to_trunc)
    for sentence in test['input_ids']: # 각 문장에 대해서
      while len(sentence) < max_len: # max_len보다 작으면
          sentence.append(0)
    test = pd.concat([test, pd.DataFrame(test['input_ids'].to_list(), columns=range(max_len))], axis=1)
    feature = test.columns.drop(['idx', 'text', 'input_ids'])
    feature = feature.drop(cols)
    pred = model.predict(test[feature])
    rf_predicts.extend(pred)

2017
2018
2019
2020


In [38]:
submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv') # sample submission 불러오기

In [39]:
submission['knowcode'] = rf_predicts

In [40]:
submission

,idx,knowcode
0,0,29401
1,1,28402
2,2,26101
3,3,121102
4,4,412003
...,...,...
35226,35244,822301
35227,35245,29303
35228,35246,833001
35229,35247,140101


In [41]:
submission.to_csv('submission.csv', index=False)

## 할일:
칼럼 매칭
칼럼이 뭔가 뒤죽박죽인 느낌임

## 내가 할일:
카테고리컬 텍스트 : 비서학, 농화학, 신문방송

그냥 텍스트 : 수송 수요가 줄어서

구분하기